In [1]:
import os
import tensorflow as tf

import matplotlib.pyplot as plt

train_dir='../dataset\\cifar_10_small\\train'
test_dir='../dataset\\cifar_10_small\\test'

train_aeroplane_dir= os.path.join(train_dir,'aeroplane')
train_bird_dir=os.path.join(train_dir,'bird')
train_car_dir= os.path.join(train_dir,'car')
train_cat_dir=os.path.join(train_dir,'cat')

test_aeroplane_dir= os.path.join(test_dir,'aeroplane')
test_bird_dir=os.path.join(test_dir,'bird')
test_car_dir= os.path.join(test_dir,'car')
test_cat_dir=os.path.join(test_dir,'cat')

print('훈련용 aeroplane 이미지 전체 개수:', len(os.listdir(train_aeroplane_dir)))
print('훈련용 bird 이미지 전체 개수:', len(os.listdir(train_bird_dir)))
print('훈련용 car 이미지 전체 개수:', len(os.listdir(train_car_dir)))
print('훈련용 cat 이미지 전체 개수:', len(os.listdir(train_cat_dir)))

print('테스트용 aeroplane 이미지 전체 개수:', len(os.listdir(test_aeroplane_dir)))
print('테스트용 bird 이미지 전체 개수:', len(os.listdir(test_bird_dir)))
print('테스트용 car 이미지 전체 개수:', len(os.listdir(test_car_dir)))
print('테스트용 cat 이미지 전체 개수:', len(os.listdir(test_cat_dir)))


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '../dataset\\cifar_10_small\\train\\aeroplane'

In [ ]:


batch_size=20
### data
# 모든 이미지를 1/255로 스케일을 조정합니다
agumentation_train_datagen = tf.keras.preprocessing.image.ImageDataGenerator( rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2, ## 가로로 이동 비율
                                    height_shift_range=0.2,  ## 세로로 이동 비율
                                    shear_range=0.2,  # 전단의 강도
                                    zoom_range=0.2, ## 확대 와 축소 범위 [1-0.2 ~ 1+0.2 ]
                                    horizontal_flip=True,)  ## 수평기준 플립
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = agumentation_train_datagen.flow_from_directory(
        directory=train_dir,         # 타깃 디렉터리
        target_size=(150, 150),      # 모든 이미지를 150 × 150 크기로 바꿉니다
        batch_size=batch_size,
        interpolation='bilinear',  ## resize시 interpolatrion 기법
        color_mode ='rgb',
        shuffle='True',
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='categorical') # categorical , sparse , input

print(train_generator.class_indices)
test_generator = test_datagen.flow_from_directory(
        directory=test_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        shuffle='True',
        interpolation='bilinear',  ## resize시 interpolatrion 기법
        color_mode='rgb',
        class_mode='categorical') #categorical

for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break



In [2]:


## back born
conv_base=tf.keras.applications.ResNet50(weights='imagenet',include_top=False)
conv_base.summary()

## 모델
input_Layer = tf.keras.layers.Input(shape=(150,150,3))
x=conv_base(input_Layer)
x=tf.keras.layers.Flatten()(x)
x= tf.keras.layers.Dense(512, activation='relu')(x)
Out_Layer= tf.keras.layers.Dense(4, activation='softmax')(x)

conv_base.trainable = False

model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()
loss_function=tf.keras.losses.categorical_crossentropy
optimize=tf.keras.optimizers.Adam(learning_rate=0.0001)
metric=tf.keras.metrics.categorical_accuracy
model.compile(loss=loss_function,
              optimizer=optimize,
              metrics=[metric])


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               26214912  
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 2052      
Total params: 49,804,676
Trainable params: 26,216,964
Non-trainable params: 23,587,712
_________________________________________________________________


In [3]:

## generator는 입력과 타싯의 배치를 끝없이 반환한다.
## 데이터가 끝없이 생성되기 때문에 모델에 하나의 에포크를 정의할때 제너레이터로부터 얼만큼 많은 샘플을 뽑을지 전달해야함
## steps_pr_epoch 가 100이면 위에서 선언된 배치 수만큼의 인풋/아웃풋데이터가 생성되어 학습이 된다.
##즉, 배치가 20이면 20의 배치인 데이터가 100번만큼 생성되어 학습한다. 즉, 20의 배치 데이터를 100번 학습완료하면 1에포크
## 단, 20의 배치데이터를 생성할때마다 랜덤적으로 생성한다.
## 일반적으로 배치사이즈/전체 데이터 길이를 steps_per_epoch로 설정한다.
history = model.fit(
      train_generator,
      steps_per_epoch=(len(os.listdir(train_aeroplane_dir)) + len(os.listdir(train_bird_dir)) + len(
         os.listdir(train_car_dir)) + len(os.listdir(train_cat_dir))) // batch_size,
      epochs=30,
      validation_data=test_generator,
      validation_freq=1)

model.save('multi_classification_augumentation_model.hdf5')


NameError: name 'train_generator' is not defined

In [ ]:

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()